<a href="https://colab.research.google.com/github/pcamarillor/O2024_ESI3914O/blob/Lab_04_Team1/Lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>